[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/taka0928eye/pytorch/blob/master/01_画像分類/load_vgg.ipynb)

In [ ]:
!git clone https://github.com/taka0928eye/pytorch.git
!cd pytorch

In [1]:
import numpy as np
import json
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torchvision
from torchvision import models, transforms

In [ ]:
use_pretrained = True # 学習済みのパラメータを使用
net = models.vgg16(pretrained=use_pretrained) # VGG-16モデルのインスタンスを生成
net.eval() # 推論モードに設定

print(net) # モデルのネットワーク構成を出力

In [ ]:
# 入力画像の前処理クラス
class BaseTransform():

    def __init__(self, resize, mean, std):
        self.base_transform = transforms.Compose([
            transforms.Resize(resize), # 短い辺の長さがresizeになる
            transforms.CenterCrop(resize), # 画像中央をresize x resizeで切り取り
            transforms.ToTensor(), # Torchテンソルに変換
            transforms.Normalize(mean, std) # 色情報の標準化
        ])
    
    def __call__(self, img):
        return self.base_transform(img)

In [ ]:
# 画像読み込み
img_path = "pytorch/01_画像分類/data/sample.jpg"
img = Image.open(img_path)

# 元画像の表示
plt.imshow(img)
plt.show()

# 画像の前処理
resize = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
transform = BaseTransform(resize, mean, std)
img_transformed = transform(img)
img_transformed = img_transformed.numpy().transpose((1, 2, 0)) # （色、高さ、幅）を（高さ、幅、色）に変換
img_transformed = np.clip(img_transformed, 0, 1) # 数値を0-1に収める

# 処理済画像の表示
plt.imshow(img_transformed)
plt.show()

In [ ]:
# 出力結果からラベルを予測する後処理クラス
class ILSVRCPredictor():
    
    def __init__(self, class_index):
        self.class_index = class_index
    
    # 確率が最も高いものを返す
    def predict_max(self, out):
        max_id = np.argmax(out.detach().numpy())
        predict_label_name = self.class_index[str(max_id)][1]
        return predict_label_name

In [ ]:
ILSVRC_class_index = json.load(open("pytorch/01_画像分類/data/imagenet_class_index.json"))
predictor = ILSVRCPredictor(ILSVRC_class_index)

img = Image.open(img_path)

transform = BaseTransform(resize, mean, std)
img_transformed = transform(img)
inputs = img_transformed.unsqueeze_(0) # バッチサイズの次元を追加

out = net(inputs)
result = predictor.predict_max(out)

print("入力画像の予測結果: ", result)

